## Métodos Numéricos II: Examen Práctica 3

#### Autor: Juan Manuel Rodríguez Gómez

In [1]:
import sympy as sp

import numpy as np

from tabulate import tabulate # Si no se encuentra el módulo, hacer "conda install tabulate" en la consola con el actual
                              # environment activado (se puede abrir desde Anaconda)

In [2]:
t, y0, z = sp.symbols('t, y_0,  z')

def f(t,y):
    return (t-y)/2 # Ecuación y'(t)=(t-y(t))/2 

In [3]:
a = 0; b = 1;   # Extremos inferior y superior del intervalo 
ya = 1          # Condición inicial del PVI
N = 10 # Como h = 0.1, tomamos N = 10 puesto que h=(b-a)/N

In [4]:
# Vamos a empezar definiendo la Ec. Dif. Ordinaria (edo) a resolver
y = sp.Function('y')
edo = y(t).diff(t)-f(t,y(t))
sp.Eq(edo,0)

Eq(-t/2 + y(t)/2 + Derivative(y(t), t), 0)

In [5]:
# La resolvemos simbólicamente mediante la orden dsolve de SymPy
edo_sol = sp.dsolve(edo)
edo_sol

Eq(y(t), C1*exp(-t/2) + t - 2)

In [6]:
# Veámos ahora cómo imponer las condiciones iniciales a través de una entrada de diccionario
ics = {y(0): y0} 
ics

{y(0): y_0}

In [7]:
# De esta manera haríamos la sustituciones correspondientes
edo_sol.rhs.subs(t, 0),edo_sol.lhs.subs(t, 0).subs(ics)

(C1 - 2, y_0)

In [8]:
CI_eq = sp.Eq(edo_sol.lhs.subs(t, 0).subs(ics),edo_sol.rhs.subs(t, 0))
CI_eq   # Esta sería finalmente la ecuación asociada a la imposición de la condición inicial del PVI

Eq(y_0, C1 - 2)

In [9]:
CI_sol = sp.solve(CI_eq) # Resolvemos mediante la orden solve y extraemos la única solución de ésta
CI_sol[0]               

{C1: y_0 + 2}

In [10]:
edo_sol   # Usaremos finalmente la solución analítica general

Eq(y(t), C1*exp(-t/2) + t - 2)

In [11]:
# Así como la obtención de las constantes de integración en función de las condiciones iniciales
edo_sol.subs(CI_sol[0])

Eq(y(t), t + (y_0 + 2)*exp(-t/2) - 2)

In [12]:
# para obtener la única solución exacta de nuestro PVI 
sol_exacta = edo_sol.rhs.subs(CI_sol[0]).subs({y0:ya})
sol_exacta

t - 2 + 3*exp(-t/2)

In [13]:
def solucion_exacta(t):
    return t-2+3*np.exp(-t/2)

In [14]:
ySolExacta = []

for h in range(0,10,1):
    ySolExacta.append(solucion_exacta(h*0.1)) 

    
ySolExacta # Valores de la Solución Exacta del PVI

[1.0,
 0.9536882735021419,
 0.9145122541078787,
 0.8821239292751735,
 0.8561922592339455,
 0.8364023492142145,
 0.8224546620451538,
 0.8140642691561406,
 0.8109601381069178,
 0.81288445486532]

In [15]:
def euler_mejorado(F,x0,y0,xfinal,N):    
    ''' método de Euler mejorado (o del punto medio) aproximado para resolver el PVI
    X,Y     = euler_mejorado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + h*F(X[n]+(h/2),Y[n]+(h/2)*F(X[n],Y[n])))
        
    return np.array(X),np.array(Y)

In [16]:
def euler_modificado(F,x0,y0,xfinal,N):    
    ''' método de Euler modificado (o de Heun) aproximado para resolver el PVI
    X,Y     = euler_modificado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + (h/2)*(F(X[n],Y[n])+F(X[n],Y[n]+h*F(X[n],Y[n]))))
        
    return np.array(X),np.array(Y)

In [17]:
xx, yEulerMejorado = euler_mejorado(f,a,ya, b, N)

yEulerMejorado # Valores de la solución aproximada del PVI con el Método de Euler Mejorado (o del Punto Medio)

array([1.        , 0.95375   , 0.91462969, 0.88229149, 0.85640478,
       0.83665505, 0.82274311, 0.81438439, 0.81130815, 0.81325688,
       0.8199856 ])

In [18]:
xx, yEulerModificado = euler_modificado(f,a,ya, b, N)

yEulerModificado # Valores de la solución aproximada del PVI con el Método de Euler Modificado

array([1.        , 0.95125   , 0.90975156, 0.87515117, 0.84711255,
       0.82531582, 0.80945667, 0.79924566, 0.79440743, 0.79468007,
       0.79981442])

In [19]:
valores = []

for i in range(0,10,1):  
    valores.append([i,ySolExacta[i], yEulerMejorado[i], yEulerModificado[i]])
    

print(tabulate(valores, headers=["i", "Solución Exacta", "Euler Mejorado (Punto Medio)", "Euler Modificado"]))

  i    Solución Exacta    Euler Mejorado (Punto Medio)    Euler Modificado
---  -----------------  ------------------------------  ------------------
  0           1                               1                   1
  1           0.953688                        0.95375             0.95125
  2           0.914512                        0.91463             0.909752
  3           0.882124                        0.882291            0.875151
  4           0.856192                        0.856405            0.847113
  5           0.836402                        0.836655            0.825316
  6           0.822455                        0.822743            0.809457
  7           0.814064                        0.814384            0.799246
  8           0.81096                         0.811308            0.794407
  9           0.812884                        0.813257            0.79468


#### Vemos que con el Método de Euler Mejorado (o del Punto Medio) obtenemos una mejor aproximación  a la solución exacta con respecto al Método de Euler Modificado (o de Heun).

In [20]:
def RK2(F,x0,y0,xfinal,N):
    ''' método de Runge-Kutta óptimo (alpha = 3/4, beta = 2/3) de m = 2 evaluaciones para resolver el PVI
    X,Y     = euler_modificado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + (h/4)*(F(X[n], Y[n]) + 3*F(X[n] + (2/3)*h, Y[n] + (2/3)*h*F(X[n],Y[n]))))
        
    return np.array(X),np.array(Y)

In [21]:
xx, yRK2 = RK2(f,a,ya, b, N)

yRK2 # Valores de la solución aproximada del PVI con el Método de Runge-Kutta óptimo de m = 2 evaluaciones

array([1.        , 0.95375   , 0.91462969, 0.88229149, 0.85640478,
       0.83665505, 0.82274311, 0.81438439, 0.81130815, 0.81325688,
       0.8199856 ])

In [22]:
valores = []

for i in range(0,10,1):  
    valores.append([i,ySolExacta[i], yEulerMejorado[i], yEulerModificado[i], yRK2[i]])
    

print(tabulate(valores, headers=["i", "Solución Exacta", "Euler Mejorado (Punto Medio)", "Euler Modificado", "RK2"]))

  i    Solución Exacta    Euler Mejorado (Punto Medio)    Euler Modificado       RK2
---  -----------------  ------------------------------  ------------------  --------
  0           1                               1                   1         1
  1           0.953688                        0.95375             0.95125   0.95375
  2           0.914512                        0.91463             0.909752  0.91463
  3           0.882124                        0.882291            0.875151  0.882291
  4           0.856192                        0.856405            0.847113  0.856405
  5           0.836402                        0.836655            0.825316  0.836655
  6           0.822455                        0.822743            0.809457  0.822743
  7           0.814064                        0.814384            0.799246  0.814384
  8           0.81096                         0.811308            0.794407  0.811308
  9           0.812884                        0.813257            0.79468 

#### Vemos que no hay mucha diferencia entre el Método de Euler Mejorado y el Método de Runge-Kutta óptimo de 2 evaluaciones.

In [23]:
errores = []

for i in range(0,10,1):  
    
    error_euler_mejorado = abs(yEulerMejorado[i]-ySolExacta[i])
    error_euler_modificado = abs(yEulerModificado[i]-ySolExacta[i])
    error_rk2 = abs(yRK2[i]-ySolExacta[i])
    
    errores.append([i, error_euler_mejorado, error_euler_modificado, error_rk2])
    

print(tabulate(errores, headers=["i", "Errores Euler Mejorado (Punto Medio)", "Errores Euler Modificado", "Errores RK2"]))

  i    Errores Euler Mejorado (Punto Medio)    Errores Euler Modificado    Errores RK2
---  --------------------------------------  --------------------------  -------------
  0                             0                            0             0
  1                             6.17265e-05                  0.00243827    6.17265e-05
  2                             0.000117433                  0.00476069    0.000117433
  3                             0.000167561                  0.00697276    0.000167561
  4                             0.000212521                  0.00907971    0.000212521
  5                             0.000252698                  0.0110865     0.000252698
  6                             0.000288451                  0.012998      0.000288451
  7                             0.000320118                  0.0148186     0.000320118
  8                             0.00034801                   0.0165527     0.00034801
  9                             0.000372421           

In [24]:
error_max_euler_mejorado = np.max(error_euler_mejorado)
error_max_euler_modificado = np.max(error_euler_modificado)
error_max_rk2 = np.max(error_rk2)


print('Error Máximo Euler Mejorado:', error_max_euler_mejorado)
print('Error Máximo Euler Modificado:', error_max_euler_modificado)
print('Error Máximo RK2:', error_max_rk2)

Error Máximo Euler Mejorado: 0.0003724208012951813
Error Máximo Euler Modificado: 0.018204384742865454
Error Máximo RK2: 0.0003724208012951813


#### Observamos que el Método de Euler Modificado es el que tiene el mayor error máximo comparado con los otros dos métodos, luego, es el que peor se aproxima (aunque tampoco se dispara mucho). Por otro lado, el Método de Euler Mejorado y el Método de RK2 tienen el mismo error máximo, luego, se aproximan númericamente igual de bien hacia la solución exacta del PVI.

#### Esto es así porque todos los métodos tienen el mismo orden, p = 2. De hecho, para determinados valores de alpha y beta en el método de Runge-Kutta de 2 evaluaciones, obtenemos precisamente los métodos de Euler Mejorado (alpha = 1, beta = 1/2) y Euler Modificado (alpha = 1/2, beta = 1).